Over a set of N sequences, with a known cleavage site for each, one can first count c(a, i), the number of 
occurrences of each amino acid a ∈ A, at every position i ∈ {−p, ..., q − 1}, relative to the corresponding 
cleavage site. Then, for each a and i, let define f(a,i) = c(a,i)/N, the observed frequency of amino acid a at the 
relative position i.

In a same way, by counting over the whole length of given sequences, one can compute the observed general background 
frequency g(a) of amino acid a in the given set, regardless of the position. However, it must be noticed that the very 
first amino acid at the beginning of a sequence is almost always an M, because it corresponds to the transcription of 
the start codon. Also, one will not count letters on this first position to avoid a bias.

These frequencies will be used as estimated probabilities to compute the probability of a given word to be 
located at a cleavage site, under an independent model. We rather use the logarithm of probabilities to go on 
additive calculations.

Then, ∀a ∈ A,∀i ∈ {-p,...,q-1}, we define s(a,i) = log(f(a,i)) - log(g(a)). Also, as zero
counts may occur, pseudo-counts [3] must be used. Finally, for any word w = a0a1 · · · a(p+q−1),
the q − 1 score defined as sum for i in [-p, ..,q-1] of s(a(p+i), i) may tell whether w is the neighborhood of a 
cleavage i=−p
site or not. A simple thresholding (to be tuned) is then enough to define a simple binary classifier.

## Import necessarry dependencies

In [1]:
import pandas as pd
import math
from auxFonctions import AminoAcid

/var/folders/wp/4cr2jl295bs4z4czxy_p6w5m0000gn/T/ipykernel_16057/407760855.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Processing the data

In [2]:
# Read data from a file into a list of entries
with open('/Users/mathiasperez/Documents/GitHub/442-2-Protein-cleavage/data/SIG_13.red', 'r') as file:
    entries = file.read().split('\n   ')


# Define a function to process each entry in the data file
def process_entry(entry):
    lines = entry.split('\n')
    protein_id, primary_structure, annotation = lines
    return {
        'Protein ID': protein_id.split()[1],
        'Primary Structure': primary_structure,
        'Annotation': annotation
    }


# Process each entry
processed_entries = [process_entry(entry) for entry in entries]

# Create a DataFrame
df = pd.DataFrame(processed_entries)

# Now you can analyze the DataFrame as needed

## Get important information our of entries 

In [3]:
# Get the position of the cleavage site
cleavage_site_position = df['Annotation'].apply(lambda x: x.find('C'))
#print("Position of the cleavage site:")
#print(cleavage_site_position)
print("Average position of the cleavage site:")
print(cleavage_site_position.mean())
print("\n")

print("The extremum position of the cleavage site:")
print(cleavage_site_position.min())
print(cleavage_site_position.max())

print("The distance from cleavage site to the C-terminal part of the amino-sequence:")
#there is always 30 amino-acids after the cleavage site
print("\n")\

# with have then p = [13, 1] and q = [1, 30]

Average position of the cleavage site:
24.04971590909091


The extremum position of the cleavage site:
13
90
The distance from cleavage site to the C-terminal part of the amino-sequence:




In [5]:
# Split the primary structure into a list of amino acids
amino_acid_seq = df['Primary Structure'].apply(lambda x: list(x))

# Create a DataFrame to store, for each primary structure, the neihborhood of the cleavage site
# The neighborhood is defined as the word of length p+q starting p letters before the cleavage site
correct_neighborhood = pd.Series(index=amino_acid_seq.index, dtype=str)
for i, seq in amino_acid_seq.items():
    correct_neighborhood[i] = ''.join(seq[cleavage_site_position[i]-2:cleavage_site_position[i]+6])

print("Neighborhood of the cleavage site:")
print(correct_neighborhood.get(0))


# for each amino acid in the sequence, replace it with the corresponding AminoAcid object
amino_acid_seq = amino_acid_seq.apply(lambda x: [AminoAcid(aa) for aa in x])

Neighborhood of the cleavage site:
IARHQQRQ


## Traitement des données 

Over a set of N sequences, with a known cleavage site for each, one can first count c(a, i), 
the number of occurrences of each amino acid a ∈ A, at every position i ∈ {−p, ..., q − 1}, 
relative to the corresponding cleavage site. 
We are facing a binary classification problem. 
Given any whole protein sequence (ai)i=0,...,l−1, and any position j, where p ≤ j ≤ l−q, 
the word aj−paj−p+1 · · · aj−1aj · · · aj+q−1 ∈ Ap+q should be enough to decide 
whether the bond at position j, between aj−1 and aj, is a cleavage site or not.

Then, for each a and i, let define f(a,i) = c(a,i)/N, 
the observed frequency of amino acid a at the relative position i.

In [6]:
#Parametres de l'étude
p = 2
q = 6

# Create a DataFrame to store the counts of each amino acid at every position relative to the cleavage site
#the cleavage site is between to aminoacids, so cleavage_site_position is the position of the first amino acid after the cleavge site
#So i need to create a dataframe with columns from -p to q without 0
amino_acid_counts = pd.DataFrame(0, index=AminoAcid.properties.keys(), columns=range(-p, q))
amino_acid_freqs = pd.DataFrame(0.0, index=AminoAcid.properties.keys(), columns=range(-p, q)) #f(a,i)
amino_acid_pseudo_counts = pd.DataFrame(0, index=AminoAcid.properties.keys(), columns=range(-p, q)) #g(a)
amino_acid_s_values = pd.DataFrame(0.0, index=AminoAcid.properties.keys(), columns=range(-p, q)) #s(a,i)


# Count the occurrences of each amino acid at every position relative to the cleavage site

for i, seq in amino_acid_seq.items():
    for j, aa in enumerate(seq):
        position = j - cleavage_site_position[i] #position of the amino acid relative to the cleavage site
        if position in amino_acid_counts.columns:
            amino_acid_counts.loc[aa.code, position] += 1

# Add pseudo-counts to avoid zero counts here pseudocount parameter is 1/len(df)
amino_acid_pseudo_counts = amino_acid_counts + 1

# Print the results
#print("Occurrences of each amino acid at every position relative to the cleavage site:")
#print(amino_acid_pseudo_counts)

# Compute the observed frequency of each amino acid at the relative position (using pseudo-counts)
for i in amino_acid_counts.index:
    for j in amino_acid_counts.columns:
        amino_acid_freqs.loc[i, j] = amino_acid_pseudo_counts.loc[i, j] / len(df)

# Compute the general background frequency of each amino acid
general_background_frequency = amino_acid_freqs.mean(axis=1)

# Compute the s value of each amino acid at every position
for i in amino_acid_counts.index:
    for j in amino_acid_counts.columns:
        amino_acid_s_values.loc[i, j] = math.log(amino_acid_freqs.loc[i, j]) - math.log(general_background_frequency[i])

## Calcul du score d'un mot :

Finally, for any word w = a0a1 · · · ap+q−1,
The q − 1 score defined as Pq−1 s(ap+i, i) may tell whether w is the neighborhood of a cleavage i=−p site or not.

In [7]:
# Define the function computing the q-1 score for a given word
def q_minus_1_score(word):
    return sum([amino_acid_s_values.loc[aa, i-p] for i, aa in enumerate(word)])

#A REDEFINIR EN FONCTION DES RESULTATS OBTENUS
threshold = 1.2715345845764872 - 2*1.5159037150732593
print(threshold)

#A simple thresholding (to be tuned) is then enough to define a simple binary classifier.
def is_cleavage_neighborhood(score):
    return score > threshold

q_minus_1_score('AAAAAAAA')

-1.7602728455700314


-2.944865582502577

## TEST DES RESULTATS OBTENUS

In [8]:
# To obtain the score of the correct neighborhoods, we apply the q-1 score function to each neighborhood
#correct_neighborhood = correct_neighborhood.apply(lambda x: [AminoAcid(aa) for aa in x])
correct_neigboorhood_score = correct_neighborhood.apply(q_minus_1_score)

print("Score of the correct neighborhoods:")
print(correct_neigboorhood_score)
print("\n")

print("Mean score of the correct neighborhoods:")
print(correct_neigboorhood_score.mean())
print("\n")

print("Standard deviation of the score of the correct neighborhoods:")
print(correct_neigboorhood_score.std())

print("Min and max values of the correct neighboorhoods score :")
print(correct_neigboorhood_score.min())
print(correct_neigboorhood_score.max())

#We now test, with the updated threshold, the performance of the classifier on the training set
# Treshold = mean - std
false_positives = correct_neigboorhood_score[correct_neigboorhood_score < threshold].count()
print("False negative out of 1005 entries:")
print(false_positives)





Score of the correct neighborhoods:
0       0.405416
1       1.402087
2       1.848045
3       2.103103
4       1.964889
          ...   
1403    3.537215
1404    1.163552
1405    3.050840
1406    2.031554
1407    1.291033
Length: 1408, dtype: float64


Mean score of the correct neighborhoods:
1.2715345845764872


Standard deviation of the score of the correct neighborhoods:
1.5159037150732593
Min and max values of the correct neighboorhoods score :
-5.410270058115765
4.481498528633624
False negative out of 1005 entries:
60
